In [2]:
import pandas as pd
import time
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes

backend = default_backend()

# Read the CSV file
df = pd.read_csv(r"C:\Users\91974\Documents\Bitcoin_tweets.csv")

def derive_aes_key(ecc_private_key, key_size):
    shared_secret = ecc_private_key.exchange(ec.ECDH(), ecc_private_key.public_key())
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=key_size // 8,
        salt=None,
        info=None,
        backend=backend
    ).derive(shared_secret)
    return derived_key

def aes_encrypt_with_ecc(df, key_size):
    ecc_private_key = ec.generate_private_key(ec.SECP256R1(), backend)
    key = derive_aes_key(ecc_private_key, key_size)
    nonce = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=backend)
    encryptor = cipher.encryptor()

    encrypted_cols = {}
    for c in df.columns:
        encrypted_cols["encrypted_" + c] = df[c].apply(lambda v: encryptor.update(str(v).encode()).decode('ISO-8859-1') if v is not None else None)

    encrypted_df = pd.concat([df, pd.DataFrame(encrypted_cols)], axis=1)
    encrypted_df["key"] = key.hex()
    encrypted_df["nonce"] = nonce.hex()
    
    return encrypted_df

def aes_decrypt_with_ecc(encrypted_df, key_size):
    key = bytes.fromhex(encrypted_df.iloc[0]["key"])
    nonce = bytes.fromhex(encrypted_df.iloc[0]["nonce"])
    cipher = Cipher(algorithms.AES(key), modes.CTR(nonce), backend=backend)
    decryptor = cipher.decryptor()

    decrypted_cols = {}
    for c in encrypted_df.columns:
        if c.startswith("encrypted_"):
            decrypted_cols["decrypted_" + c.replace("encrypted_", "")] = encrypted_df[c].apply(lambda v: decryptor.update(v.encode('ISO-8859-1')).decode() if v is not None else None)

    decrypted_df = pd.DataFrame(decrypted_cols)
    return decrypted_df

key_sizes = [128, 192, 256]  # Valid key sizes for AES
for key_size in key_sizes:
    print(f"\nTesting with key size {key_size} bits\n")

    # Encrypt the dataset using AES with the specified key size
    start_time = time.time()
    df_encrypted = aes_encrypt_with_ecc(df, key_size)
    end_time = time.time()
    encryption_time = end_time - start_time
    print(f"Overall encryption time: {encryption_time:.4f} seconds\n")
    print(df_encrypted.head())

    # Decrypt the dataset using AES with the specified key size
    start_time = time.time()
    df_decrypted = aes_decrypt_with_ecc(df_encrypted, key_size)  # Use pandas DataFrame
    end_time = time.time()
    decryption_time = end_time - start_time
    print(f"Overall decryption time: {decryption_time:.4f} seconds\n")
    print(df_decrypted.head())  # Display first few rows of the decrypted DataFrame





Testing with key size 128 bits

Overall encryption time: 14.8865 seconds

                                  user_name    user_location  \
0                             DeSota Wilson      Atlanta, GA   
1                                  CryptoND              NaN   
2                                 Tdlmatias  London, England   
3                      Crypto is the future              NaN   
4  Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader           Europa   

                                    user_description  \
0  Biz Consultant, real estate, fintech, startups...   
1  😎 BITCOINLIVE is a Dutch platform aimed at inf...   
2  IM Academy : The best #forex, #SelfEducation, ...   
3  I will post a lot of buying signals for BTC tr...   
4  Co-founder @RENJERJerky | Forbes 30Under30 | I...   

                                 encrypted_user_name encrypted_user_location  \
0                                      øaà¬Ð%             £¿ ÌFVÑ   
1                                          